# Overview

We measure the F1, recall, and precision scores of our GPT-generated data labels when matched exactly against human labels.

# Env Setup

In [ ]:
!pip install transformers seqeval[gpu]

In [ ]:
import csv
import json
import pandas as pd
import numpy as np
import re
import string
from collections import Counter
import os
from google.colab import drive
import time
import ast
import random
from collections import defaultdict
from seqeval.metrics import accuracy_score
from seqeval.metrics import classification_report
from seqeval.metrics import f1_score

In [ ]:
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd drive/MyDrive/'6.8611 Research Project'/'Colab Notebooks'

[Errno 2] No such file or directory: 'drive/MyDrive/6.8611 Research Project/Colab Notebooks'
/content/drive/.shortcut-targets-by-id/1vdEcgdXIfpnlORVlPsJtHUmKXSAqr69R/6.8611 Research Project/Colab Notebooks


In [ ]:
ls

 BC5CDR-D_devel_1.csv                Intrinsic_token.ipynb
 BC5CDR-D_devel_2.csv                llm-annotations/
 Data-cleaning.ipynb                ' NER with BERT.ipynb'
 devel_gpt_generated_datasets/       openai-test.ipynb
 Fine-Tuning-Few-Shot.ipynb          retry_prompts.gsheet
 Fine-Tuning-Human-Annotated.ipynb   RW-Fine-Tuning-Human-Annotated.ipynb
 Fine-Tuning-One-Shot.ipynb          sft_training_data/
 Fine-Tuning-Zero-Shot.ipynb         TEST_LABEL_BUGS.ipynb
 GPT-Finetuning.ipynb                tokens_labels.csv
 intrinsic_data/                     Untitled
 intrinsic_eval_entity.ipynb         zero-shot-bc5cdr-chem.pynb
 intrinsic_eval.ipynb               'zero_shot[FASTER].ipynb'
'Intrinsic Eval Precision.ipynb'     zero-shot.pynb
 Intrinsic_one.ipynb


# Data Loading

In [ ]:
# SET PARAMS

dataset = 'JNLPBA'
prompt_type = 'O'

In [ ]:
file_path = f'intrinsic_data/{dataset}-{prompt_type}S.csv'

In [ ]:
data = pd.read_csv(file_path)

# Exact Match Intrinsic Evaluation

In [ ]:
def relabel(df):
  for i, row in df.iterrows():
    if row['label'] not in {'B', 'I', 'O'}:
      print('yo')
      if row['label'][0] in {'B', 'I', 'O'}:
        df.loc[i, 'label'] = row['label'][0]
      else:
        df.loc[i, 'label'] = 'O'
    if row['label_gpt'] not in {'B', 'I', 'O'}:
      if isinstance(row['label_gpt'], str) and row['label_gpt'][0] in {'B', 'I', 'O'}:
        df.loc[i, 'label_gpt'] = row['label_gpt'][0]
      else:
        df.loc[i, 'label_gpt'] = 'O'
  return df


In [ ]:
types = {'Z': 'Zero Shot', 'O': 'One Shot', 'F': 'Few Shot'}

def get_report(dataset, prompt_type):
  file_path = f'intrinsic_data/{dataset}-{prompt_type}S.csv'
  data = pd.read_csv(file_path)
  data = relabel(data)
  labels = [data['label'].to_list()]
  predictions = [data['label_gpt'].to_list()]
  print('-----------------------------------------')
  print(dataset +' '+ types[prompt_type]+ ': ')
  print()
  print(classification_report(labels, predictions))
  print('-----------------------------------------')
  print()

In [ ]:
datasets = ['NCBI', 'BC2GM', 'JNLPBA', 'BC5CDR-disease', 'BC5CDR-chem']
prompt_types = ['Z', 'O', 'F']
for dataset in datasets:
  for prompt_type in prompt_types:
    get_report(dataset, prompt_type)

-----------------------------------------
NCBI Zero Shot: 

              precision    recall  f1-score   support

           _       0.22      0.46      0.30       787

   micro avg       0.22      0.46      0.30       787
   macro avg       0.22      0.46      0.30       787
weighted avg       0.22      0.46      0.30       787

-----------------------------------------

-----------------------------------------
NCBI One Shot: 

              precision    recall  f1-score   support

           _       0.25      0.59      0.35       787

   micro avg       0.25      0.59      0.35       787
   macro avg       0.25      0.59      0.35       787
weighted avg       0.25      0.59      0.35       787

-----------------------------------------

-----------------------------------------
NCBI Few Shot: 

              precision    recall  f1-score   support

           _       0.27      0.42      0.33       787

   micro avg       0.27      0.42      0.33       787
   macro avg       0.27   